In [11]:
import subprocess
import requests
# 사이트에서 이미지 가져오기 위해서
import xml.etree.ElementTree as ET
#나무 구조 형태로 변환하는 모듈
import openpyxl
from openpyxl.drawing.image import Image as ExcelImage
from openpyxl.styles import Alignment, Font, PatternFill
from io import BytesIO
# 컴퓨터의 코드를 사람이 인지할 수 있는 이미지로 변환

def px_to_row_height(px: int) -> float :
    return px * 0.75
    
def px_to_col_width(px: int) -> float :
    return (px - 14) / 7

BASE_URL = "https://apis.data.go.kr/B551011/GoCamping/basedList"
API_KEY = "42a5635aa5c75221fc6de35f3eb03327ef3c1c8502f63839d7084ddb87d1a3b9"

url = f"{BASE_URL}?numOfRows=50&pageNo=1&MobileOS=ETC&MobileApp=camping&serviceKey={API_KEY}"

result = subprocess.run(["curl", "-k", url], capture_output=True, text=True, encoding="utf-8") 
#curl: clinter for url, -k : 오류가 나면 종료 시켜라, capture_output : 오류 나면 보여줘, text : 텍스트로

root = ET.fromstring(result.stdout)

wb = openpyxl.Workbook()
ws = wb.active
ws.title = "캠핑장 정보"
headers = ["아이디", "캠핑장명", "이미지", "지역", "주소", "홈페이지"]
ws.append(headers)

# 이미지 사이즈
IMG_W_PX = 100
IMG_H_PX = 75
# 엑셀에 삽입될 사이즈
ROW_H_PT = px_to_row_height(IMG_H_PX)
COL_W_PT = px_to_col_width(IMG_W_PX)

cell_align = Alignment(vertical="center", horizontal="left", wrap_text=True)
# wrap_text : 자료값의 문자가 셀의 넓이를 넘어갈 경우 줄바꿈

header_fill = PatternFill("solid", fgColor="F2F2F2")
header_font = Font(bold=True)

for col in range(1, len(headers) + 1) :
    c = ws.cell(row=1, column=col)
    c.alignment = Alignment(vertical="center", horizontal="center")
    c.fill = header_fill
    c.font = header_font
    
# solid : 단색 / gradation

items = root.findall(".//item")
# '.' : (최상위) '//' :아래 있는 item 찾아와줘

row = 2

for item in items :
    contentId = item.findtext("contentId", default="")
    facltNm = item.findtext("facltNm", default="")
    firstImageUrl = item.findtext("firstImageUrl", default="")
    doNm = item.findtext("doNm", default="")
    addr1 = item.findtext("addr1", default="")
    homepage = item.findtext("homepage", default="")

    ws.append([contentId, facltNm, "", doNm, addr1, homepage])

    ws.row_dimensions[row].height = ROW_H_PT

    for col in range(1, 7) :
        ws.cell(row=row, column=col).alignment = cell_align

    if firstImageUrl :
        try :
            img_data = requests.get(firstImageUrl, timeout=10).content
            img_file = BytesIO(img_data)
            img = ExcelImage(img_file)
            img.width = IMG_W_PX
            img.height = IMG_H_PX
            ws.add_image(img, f"C{row}")
        except Exception as e :
            print(f"[경고] 이미지 다운로드 실패 : {firstImageUrl} -> {e}")

    row += 1

ws.column_dimensions["A"].width = 12
ws.column_dimensions["B"].width = 30
ws.column_dimensions["C"].width = COL_W_PT
ws.column_dimensions["D"].width = 15
ws.column_dimensions["E"].width = 50
ws.column_dimensions["F"].width = 50

file_name = "camping_info.xlsx"

ws.freeze_panes = "A2"
# 엑셀의 첫번째행 고정

wb.save(file_name)
print(f"😁{file_name} 파일 저장 완료")

# xml 에서 findtext("") 이 태그를 가져와라
# deault="" 값이 없을 경우 빈칸
# contentId : 데이터의 고유한 식별값은 필요하지 않더라도 가져오는 것이 현명하다 (추후 변수 대비)
# img_data : 컴퓨터가 이미지를 이해할 수 있는 형태(스트림코드), img_file : 사람이 이해할 수 있는 형태
# 이미지의 단위는 픽셀, 엑셀은 픽셀 아닌 넓이와 높이로 인식. 따라서 IMG_W_PX = 100, IMG_H_PX = 75 설정 후 img.width = IMG_W_PX

😁camping_info.xlsx 파일 저장 완료
